In [ ]:
from PIL import Image
import cv2

##Setting Up The YOLOv5 Environment

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies



In [ ]:
#installing for google colab GPU use
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd /content/yolov5

In [ ]:
!ls

In [ ]:
import torch
from IPython.display import Image

print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

### Downloading the data

In [ ]:
!curl -L "https://app.roboflow.com/ds/YRDeArnNYZ?key=7zCCi2Zm26" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Path to train dataset - /content/yolov5/train 

Path to test dataset - /content/yolov5/test

## Define Model Configuration and Architecture

In [ ]:
%cat data.yaml


Path to the train and test dataset is not correct. We need to edit it in the next steps

In [ ]:
# define number of classes based on YAML
# data.yaml contains number of classes and labels required
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# Change the data configuration for right path to the dataset
%%writetemplate /content/yolov5/data.yaml

train: ./train/images
val: ./valid/images

nc: 2
names: ['FishRaysKelp', 'Kelp']

In [ ]:
%cat data.yaml

In [ ]:
with open(r'data.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    labels_list = yaml.load(file, Loader=yaml.FullLoader)

    label_names = labels_list['names']

In [ ]:
print("Number of Classes: {}, Labels: {}".format(num_classes,label_names))

In [ ]:
# Use yolov5l.yaml configuration for training.
%cat /content/yolov5/models/yolov5l.yaml

In [ ]:
# Change configuration so that it corresponds to number of classes
%%writetemplate /content/yolov5/models/custom_yolov5l.yaml

# Parameters
nc: num_classses  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Train Custom YOLOv5





In [ ]:
import os
os.chdir('/content/yolov5')

In [ ]:
# wandb portion is optional
!pip install wandb
import wandb

In [ ]:
# train yolov5l on object detection data for 100 epochs with batch size of 80
# images are pre-processed to 416 x 416 size
# data.yaml contains location of train and validation data
# configuration of neural network is in custom_yolov5l.yaml
# weights stored at /content/yolov5/runs/exp2/weights/best.pt
# time performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 64 --epochs 100 --data './data.yaml' --cfg ./models/custom_yolov5l.yaml --weights '' --device 0

## Evaluate Custom YOLOv5 Performance

In [ ]:
# start tensorboard
# launch after starting training
# logs saved in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs

## Visualize Training Data with Labels



In [ ]:
# display ground truth data
# ground truth/training data available at /content/yolov5/runs/train/exp2/val_batch0_labels.jpg 
print("GROUND TRUTH TRAINING DATA:")
im = Image.open('/content/yolov5/runs/train/exp2/val_batch0_labels.jpg')
Image.Image.show(im)

In [ ]:
# print out augmented training data
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
im = Image.open('/content/yolov5/runs/train/exp2/train_batch0.jpg')
Image.Image.show(im)

# Run Inference With Trained Weights

In [ ]:
# use the best weights
# final weights stored at /content/yolov5/runs/train/exp2/weights/best.pt
%cd /content/yolov5/
!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 416 --conf 0.4 --source ./test/images

### Check the output

In [ ]:
# display inference on all test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## Export Trained Weights for Future Inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cp /content/yolov5/runs/train/exp2/weights/best.pt /content/gdrive/My\ Drive

# Split Test Video into Frames and Run Inference with Trained Weights

In [ ]:
# Change video path as needed
vidObj = cv2.VideoCapture('/content/gdrive/My Drive/KelpandFish_short.mp4')

numFrames = 0
success = 1

# create Drive directory to store files before running this chunk of code
# rename 'FishKelpFrames' to desired directory name

while success:
  success, image = vidObj.read()

  if(success == 1):
    newImg = cv2.resize(image, (416, 416))
    cv2.imwrite("/content/gdrive/MyDrive/FishKelpFrames/frame%s.jpg" % str(numFrames).zfill(4), newImg)
    numFrames += 1


In [ ]:
# use the best weights
# final weights stored at /content/yolov5/runs/train/exp2/weights/best.pt
%cd /content/yolov5/
#!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/FishKelpFrames
!python detect.py --weights /content/gdrive/MyDrive/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/FishKelpFrames

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp5/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Create Video from Detected Images

In [ ]:
frameSize = (416, 416)
out = cv2.VideoWriter('/content/gdrive/MyDrive/predictionVideo.avi', cv2.VideoWriter_fourcc(*'DIVX'), 60, frameSize)

for filename in glob.glob('/content/yolov5/runs/detect/exp5/*.jpg'):
  img = cv2.imread(filename)
  out.write(img)

out.release()

# Export Predictions

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')

%cp -r /content/yolov5 /content/gdrive/MyDrive/Augmented\ Resized\ YOLOv5

# Try With A Different Video

In [ ]:
vidObj = cv2.VideoCapture('/content/gdrive/My Drive/Fish2_short.mp4')

numFrames = 0
success = 1

while success:
  success, image = vidObj.read()

  if(success == 1):
    newImg = cv2.resize(image, (416, 416))
    cv2.imwrite("/content/gdrive/MyDrive/FishKelpFrames2/frame%s.jpg" % str(numFrames).zfill(4), newImg)
    numFrames += 1

In [ ]:
# use the best weights!
# Final weights will be by-default stored at /content/yolov5/runs/train/exp2/weights/best.pt
%cd /content/yolov5/
#!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/FishKelpFrames
!python detect.py --weights /content/gdrive/MyDrive/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/FishKelpFrames2

In [ ]:
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp4/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Try With Yet One More Video